<a id='top'></a>

# Limpar e Estruturar os Formulários de Informações Trimestrais (ITR)

# CONTENTS

* [Imports](#imports) 
* [Loading Dataset](#loading_dataset)

<a id='imports'></a>
### Imports

In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from unidecode import unidecode

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

<a id='loading_dataset'></a>
### Load dataset

In [1]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
os.getpwd()

In [ ]:
#df_cn = df.drop_duplicates(['CNPJ_CIA'])
#df_cn[['CNPJ_CIA', 'DENOM_CIA']].to_csv('../data_processed/cnpj_name.csv', encoding = "ISO-8859-1", index=False)

In [ ]:
#Abrir arquivo .csv com cnpj e ticker
path = '../data_processed/stock_info_by_cnpj.csv'
df_cnpj_ticker = pd.read_csv(path, sep=',', encoding = "ISO-8859-1")

In [ ]:
# Balanço Patrimonial Ativo (BPA)
years = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

itr_names = ['bpa', 'bpp', 'dre', 'dfc_mi', 'dva']
#itr_names = ['dre']

for itr_name in itr_names:

    for year in years:

        path = '../data/itr_cia_aberta_'+str(year)+'/itr_cia_aberta_'+itr_name+'_con_'+str(year)+'.csv'
        df = pd.read_csv(path, sep=';', encoding = "ISO-8859-1")

        # Remover todas as linhas com valor 'PENÚLTIMO' em 'ORDEM_EXERC'
        df = df[df['ORDEM_EXERC'] != 'PENÚLTIMO']

        #Deletar colunas não usadas
        df = df.drop(['VERSAO', 'GRUPO_DFP', 'ORDEM_EXERC', 'DT_FIM_EXERC', 'DS_CONTA'], axis=1)

        #Montar dataframe por ano
        df_result = pd.DataFrame({'A' : []})

        # Criar um data frame por cnpj
        cnpj_uniq = df['CNPJ_CIA'].unique()

        count = 1

        for cnpj in cnpj_uniq:

            print(count)
            print(cnpj)    
            count = count + 1

            aux = df[df['CNPJ_CIA'] == cnpj]
            cd_cvm = aux['CD_CVM'].iloc[0]
            dt_refer = aux['DT_REFER'].unique()

            #Verificar se o cnpj está listado na B3
            idx_cnpj = df_cnpj_ticker.index[df_cnpj_ticker['cnpj'] == cnpj].tolist()
            if not idx_cnpj:
                print(cnpj + ' não existe')
                print("")
            else:        
                #Obter ticker
                ticker = df_cnpj_ticker.iloc[idx_cnpj]['on_ticker'].tolist()[0]
                print('Ticker: ' + ticker)
                print("")

                for dt in dt_refer:

                    aux_refer = aux[aux['DT_REFER'] == dt]
                    aux_t = aux_refer.T.reset_index()        
                    idx = aux_t[aux_t['index'] == 'CD_CONTA'].index
                    aux_t.columns = aux_t[aux_t['index'] == 'CD_CONTA'].iloc[0]
                    aux_t = aux_t.drop([idx[0]])

                    ds_unique = aux_t['CD_CONTA'].unique()
                    i = 1
                    for ds in ds_unique: 
                        aux_t.insert((i-1), ds, aux_t[aux_t['CD_CONTA'] == ds].iloc[0][i])
                        i = i + 1                

                    #Deixar somente a última linha
                    n_row = aux_t.shape[0] - 1
                    aux_t.drop(aux_t.head(n_row).index, inplace=True) # drop first n rows                                

                    #Ordenar colunas
                    aux_t = aux_t.sort_index(axis=1)                                

                    #Converter para lower case
                    aux_t.columns = map(str.lower, aux_t.columns)                                

                    #Converter no formato unicode
                    j = 0
                    new_cols = []
                    for col in aux_t.columns:
                        new_cols = np.append(new_cols, unidecode(col))
                        j = j + 1

                    aux_t.columns = new_cols                                

                    #Remover colunas duplicadas                
                    aux_t = aux_t.loc[:,~aux_t.columns.duplicated()]                                

                    #Deletar coluna 'VL_CONTA'
                    aux_t.drop(['vl_conta', 'cd_conta'], axis=1, inplace=True)                                

                    #Adcionar coluna 'ticker'
                    aux_t['ticker'] = ticker                

                    if df_result.empty:
                        df_result = aux_t.copy()
                    else:
                        df_result = pd.concat([df_result,aux_t], axis=0, ignore_index=True)

        #Ordenar por data ('dt_refer')
        df_result.sort_values(by=['dt_refer', 'ticker'], inplace=True, ignore_index=True)

        #Reorganizar algumas colunas
        cols = df_result.columns.tolist()
        cols.insert(0, cols.pop(cols.index("dt_refer")))
        cols.insert(1, cols.pop(cols.index("cnpj_cia")))
        cols.insert(2, cols.pop(cols.index("cd_cvm")))
        cols.insert(3, cols.pop(cols.index("denom_cia")))
        cols.insert(4, cols.pop(cols.index("ticker")))
        df_result = df_result[cols]

        #Salvar dataframe em um arquivo .csv
        df_result.to_csv('../data_processed/'+str(year)+'/'+itr_name+'_'+str(year)+'.csv', encoding='utf-8', index=False)

1
00.001.180/0001-26
Ticker: ELET3

2
00.070.698/0001-11
Ticker: CEBR3

3
00.095.147/0001-02
00.095.147/0001-02 não existe

4
00.108.786/0001-65
00.108.786/0001-65 não existe

5
00.249.786/0001-85
00.249.786/0001-85 não existe

6
00.354.053/0001-00
00.354.053/0001-00 não existe

7
00.622.416/0001-41
Ticker: STKF3

8
00.743.065/0001-27
Ticker: LTEL3B

9
00.776.574/0001-56
Ticker: BTOW3

10
00.864.214/0001-06
Ticker: ENGI3

11
00.924.429/0001-75
00.924.429/0001-75 não existe

12
00.994.533/0001-36
00.994.533/0001-36 não existe

13
01.027.058/0001-91
01.027.058/0001-91 não existe

14
01.083.200/0001-18
Ticker: NEOE3

15
01.109.184/0001-95
01.109.184/0001-95 não existe

16
01.545.826/0001-07
Ticker: GFSA3

17
01.547.749/0001-16
01.547.749/0001-16 não existe

18
01.608.571/0001-76
01.608.571/0001-76 não existe

19
01.772.313/0001-20
01.772.313/0001-20 não existe

20
01.832.635/0001-18
01.832.635/0001-18 não existe

21
01.838.723/0001-27
Ticker: BRFS3

22
01.917.705/0001-30
01.917.705/0001-3

186
14.998.371/0001-19
Ticker: JMCD3

187
15.101.405/0001-93
Ticker: CORR3

188
15.115.504/0001-24
15.115.504/0001-24 não existe

189
15.141.799/0001-03
Ticker: FESA3

190
15.144.017/0001-90
Ticker: CSAB3

191
16.234.171/0001-15
16.234.171/0001-15 não existe

192
16.404.287/0001-55
Ticker: SUZB3

193
16.590.234/0001-76
Ticker: ARZZ3

194
16.614.075/0001-00
Ticker: DIRR3

195
16.670.085/0001-55
Ticker: RENT3

196
17.155.730/0001-64
Ticker: CMIG3

197
17.159.005/0001-64
17.159.005/0001-64 não existe

198
17.167.396/0001-69
17.167.396/0001-69 não existe

199
17.193.806/0001-46
Ticker: BRGE3

200
17.245.234/0001-00
Ticker: CEDO3

201
17.281.106/0001-03
Ticker: CSMG3

202
19.443.985/0001-58
19.443.985/0001-58 não existe

203
19.526.748/0001-50
Ticker: CATA3

204
21.255.567/0001-89
Ticker: CTSA3

205
22.164.990/0001-36
22.164.990/0001-36 não existe

206
22.266.175/0001-88
22.266.175/0001-88 não existe

207
22.677.520/0001-76
Ticker: CTNM3

208
22.770.366/0001-82
22.770.366/0001-82 não existe

7
00.743.065/0001-27
Ticker: LTEL3B

8
00.776.574/0001-56
Ticker: BTOW3

9
00.864.214/0001-06
Ticker: ENGI3

10
00.924.429/0001-75
00.924.429/0001-75 não existe

11
00.994.533/0001-36
00.994.533/0001-36 não existe

12
01.027.058/0001-91
01.027.058/0001-91 não existe

13
01.083.200/0001-18
Ticker: NEOE3

14
01.545.826/0001-07
Ticker: GFSA3

15
01.608.571/0001-76
01.608.571/0001-76 não existe

16
01.772.313/0001-20
01.772.313/0001-20 não existe

17
01.832.635/0001-18
01.832.635/0001-18 não existe

18
01.838.723/0001-27
Ticker: BRFS3

19
01.917.705/0001-30
01.917.705/0001-30 não existe

20
01.917.818/0001-36
01.917.818/0001-36 não existe

21
01.938.783/0001-11
Ticker: PEAB3

22
01.957.772/0001-89
Ticker: OPTS3B

23
01.971.614/0001-83
Ticker: ECPR3

24
02.105.040/0001-23
02.105.040/0001-23 não existe

25
02.107.946/0001-87
02.107.946/0001-87 não existe

26
02.149.205/0001-69
Ticker: PSSA3

27
02.193.750/0001-52
Ticker: GPCP3

28
02.302.101/0001-42
Ticker: EMAE4

29
02.312.604/0001-07
02.31

193
17.245.234/0001-00
Ticker: CEDO3

194
17.281.106/0001-03
Ticker: CSMG3

195
19.526.748/0001-50
Ticker: CATA3

196
21.255.567/0001-89
Ticker: CTSA3

197
22.164.990/0001-36
22.164.990/0001-36 não existe

198
22.266.175/0001-88
22.266.175/0001-88 não existe

199
22.677.520/0001-76
Ticker: CTNM3

200
22.770.366/0001-82
22.770.366/0001-82 não existe

201
24.962.466/0001-36
24.962.466/0001-36 não existe

202
25.329.319/0001-96
25.329.319/0001-96 não existe

203
28.500.320/0001-20
28.500.320/0001-20 não existe

204
29.780.061/0001-09
Ticker: SCAR3

205
29.978.814/0001-87
Ticker: SULA3

206
30.540.991/0001-66
Ticker: HAGA3

207
32.681.371/0001-72
32.681.371/0001-72 não existe

208
33.000.167/0001-01
Ticker: PETR3

209
33.009.911/0001-39
33.009.911/0001-39 não existe

210
33.014.556/0001-96
Ticker: LAME3

211
33.035.536/0001-00
33.035.536/0001-00 não existe

212
33.039.223/0001-11
33.039.223/0001-11 não existe

213
33.041.260/0652-90
Ticker: VVAR3

214
33.042.730/0001-04
Ticker: CSNA3

215


Ticker: TIMP3

37
02.558.124/0001-12
02.558.124/0001-12 não existe

38
02.558.157/0001-62
Ticker: VIVT3

39
02.591.787/0001-39
02.591.787/0001-39 não existe

40
02.604.860/0001-60
02.604.860/0001-60 não existe

41
02.762.113/0001-50
02.762.113/0001-50 não existe

42
02.762.115/0001-49
Ticker: MMXM3

43
02.762.121/0001-04
Ticker: STBP3

44
02.800.026/0001-40
Ticker: COGN3

45
02.808.708/0001-07
02.808.708/0001-07 não existe

46
02.846.056/0001-97
Ticker: CCRO3

47
02.860.694/0001-62
Ticker: SHOW3

48
02.916.265/0001-60
Ticker: JBSS3

49
02.919.555/0001-67
02.919.555/0001-67 não existe

50
02.932.074/0001-91
Ticker: HYPE3

51
02.950.811/0001-89
Ticker: PDGR3

52
02.998.609/0001-27
02.998.609/0001-27 não existe

53
02.998.611/0001-04
Ticker: TRPL3

54
03.014.553/0001-91
Ticker: TPIS3

55
03.220.438/0001-73
Ticker: ESCE3

56
03.378.521/0001-75
Ticker: LIGT3

57
03.601.314/0001-38
03.601.314/0001-38 não existe

58
03.616.277/0001-31
03.616.277/0001-31 não existe

59
03.758.318/0001-24
Ticke

...

34
02.846.056/0001-97
Ticker: CCRO3

35
02.860.694/0001-62
Ticker: SHOW3

36
02.916.265/0001-60
Ticker: JBSS3

37
02.919.555/0001-67
02.919.555/0001-67 não existe

38
02.932.074/0001-91
Ticker: HYPE3

39
02.950.811/0001-89
Ticker: PDGR3

40
02.998.301/0001-81
Ticker: GEPA3

41
02.998.611/0001-04
Ticker: TRPL3

42
03.014.553/0001-91
Ticker: TPIS3

43
03.220.438/0001-73
Ticker: ESCE3

44
03.378.521/0001-75
Ticker: LIGT3

45
03.601.314/0001-38
03.601.314/0001-38 não existe

46
03.758.318/0001-24
Ticker: IVPR3

47
03.818.379/0001-30
03.818.379/0001-30 não existe

48
03.847.461/0001-92
Ticker: BRAP3

49
03.853.896/0001-40
Ticker: MRFG3

50
03.953.509/0001-47
Ticker: CPFG3

51
03.983.431/0001-03
Ticker: ENBR3

52
04.031.960/0001-70
04.031.960/0001-70 não existe

53
04.032.433/0001-80
Ticker: ATMP3

54
04.065.791/0001-99
Ticker: SQIA3

55
04.128.563/0001-10
Ticker: TIET3

56
04.149.454/0001-80
Ticker: ECOR3

57
04.423.567/0001-21
Ticker: ENEV3

58
04.437.534/0001-30
Ticker: UNID3

59
04.752.9

234
58.430.828/0001-60
58.430.828/0001-60 não existe

235
59.105.999/0001-86
Ticker: WHRL3

236
60.398.369/0004-79
Ticker: PMAM3

237
60.476.884/0001-87
Ticker: LEVE3

238
60.500.139/0001-26
60.500.139/0001-26 não existe

239
60.543.816/0001-93
Ticker: JPSA3

240
60.637.238/0001-54
Ticker: JBDU3

241
60.643.228/0001-21
60.643.228/0001-21 não existe

242
60.651.809/0001-05
Ticker: NEMO3

243
60.730.348/0001-66
Ticker: MSPA3

244
60.840.055/0001-31
Ticker: FLRY3

245
60.851.615/0001-53
Ticker: BDLL3

246
60.872.504/0001-23
Ticker: ITUB3

247
60.894.730/0001-05
Ticker: USIM3

248
61.022.042/0001-18
Ticker: CALI3

249
61.065.298/0001-02
Ticker: MGEL3

250
61.065.751/0001-80
Ticker: RSID3

251
61.067.161/0001-97
61.067.161/0001-97 não existe

252
61.079.117/0001-05
Ticker: ALPA3

253
61.080.313/0001-91
61.080.313/0001-91 não existe

254
61.082.004/0001-50
Ticker: ESTR3

255
61.088.894/0001-08
Ticker: CAMB3

256
61.092.037/0001-81
61.092.037/0001-81 não existe

257
61.156.113/0001-75
Ticker:

115
08.884.213/0001-35
08.884.213/0001-35 não existe

116
08.926.302/0001-05
Ticker: DMMO3

117
08.935.054/0001-50
08.935.054/0001-50 não existe

118
09.041.168/0001-10
Ticker: LOGG3

119
09.054.385/0001-44
09.054.385/0001-44 não existe

120
09.112.685/0001-32
Ticker: OSBX3

121
09.149.503/0001-06
Ticker: OMGE3

122
09.288.252/0001-32
Ticker: ANIM3

123
09.295.063/0001-97
Ticker: TECN3

124
09.305.994/0001-29
Ticker: AZUL4

125
09.346.601/0001-25
Ticker: B3SA3

126
09.611.768/0001-76
Ticker: INNT3

127
10.215.988/0001-60
Ticker: LCAM3

128
10.345.009/0001-98
Ticker: ADHM3

129
10.629.105/0001-68
Ticker: PRIO3

130
10.760.260/0001-19
Ticker: CVCB3

131
10.851.805/0001-00
Ticker: FLEX3

132
11.102.250/0001-59
11.102.250/0001-59 não existe

133
11.284.204/0001-18
11.284.204/0001-18 não existe

134
11.395.624/0001-71
11.395.624/0001-71 não existe

135
11.669.021/0001-10
Ticker: ENAT3

136
11.721.921/0001-60
Ticker: APER3

137
11.992.680/0001-93
Ticker: QUAL3

138
12.049.631/0001-84
Ticker:

316
90.400.888/0001-42
Ticker: SANB3

317
90.441.460/0001-48
Ticker: UCAS3

318
91.333.666/0001-17
Ticker: RCSL3

319
91.495.499/0001-00
Ticker: STTR3

320
91.983.056/0001-69
Ticker: KEPL3

321
92.012.467/0001-70
Ticker: CGRA3

322
92.660.570/0001-26
Ticker: LUXM3

323
92.665.611/0001-77
Ticker: PNVL3

324
92.690.783/0001-09
Ticker: GOAU3

325
92.693.019/0001-89
Ticker: PATI3

326
92.715.812/0001-31
Ticker: CEED3

327
92.754.738/0001-62
Ticker: LREN3

328
92.781.335/0001-02
Ticker: TASA3

329
92.791.243/0001-03
Ticker: RANI3

330
97.837.181/0001-47
Ticker: DTEX3

1
00.001.180/0001-26
Ticker: ELET3

2
00.070.698/0001-11
Ticker: CEBR3

3
00.359.742/0001-08
Ticker: ATOM3

4
00.622.416/0001-41
Ticker: STKF3

5
00.776.574/0001-56
Ticker: BTOW3

6
00.864.214/0001-06
Ticker: ENGI3

7
01.083.200/0001-18
Ticker: NEOE3

8
01.107.327/0001-20
Ticker: BBML3

9
01.545.826/0001-07
Ticker: GFSA3

10
01.838.723/0001-27
Ticker: BRFS3

11
01.917.818/0001-36
01.917.818/0001-36 não existe

12
01.938.783/00

33.541.368/0001-16 não existe

189
33.592.510/0001-54
Ticker: VALE3

190
33.611.500/0001-19
Ticker: GGBR3

191
33.839.910/0001-11
Ticker: VIVA3

192
33.958.695/0001-78
Ticker: UNIP3

193
34.274.233/0001-02
Ticker: BRDT3

194
35.791.391/0001-94
Ticker: QUSW3

195
36.542.025/0001-64
Ticker: BRQB3

196
42.150.391/0001-70
Ticker: BRKM3

197
42.278.291/0001-24
Ticker: LOGN3

198
42.278.473/0001-03
Ticker: WIZS3

199
42.331.462/0001-31
Ticker: BTTL3

200
42.771.949/0001-35
Ticker: AALR3

201
43.185.362/0001-07
Ticker: IGBR3

202
43.470.988/0001-65
Ticker: EVEN3

203
45.242.914/0001-05
Ticker: CEAB3

204
45.453.214/0001-51
Ticker: PFRM3

205
45.987.245/0001-92
Ticker: BAHI3

206
47.508.411/0001-56
Ticker: PCAR3

207
47.960.950/0001-21
Ticker: MGLU3

208
49.263.189/0001-02
Ticker: HBOR3

209
49.669.856/0001-43
Ticker: LLIS3

210
50.564.053/0001-03
Ticker: BOBR3

211
50.746.577/0001-15
Ticker: CSAN3

212
50.926.955/0001-42
Ticker: VULC3

213
51.128.999/0001-90
Ticker: NUTR3

214
51.218.147/0001

In [ ]:
years = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

itr_names = ['bpa', 'bpp', 'dre', 'dfc_mi', 'dva']
#itr_names = ['dre']

for itr_name in itr_names:

    df_result = pd.DataFrame({'A' : []})

    for year in years:

        path = '../data/itr_cia_aberta_'+str(year)+'/itr_cia_aberta_'+itr_name+'_con_'+str(year)+'.csv'
        df = pd.read_csv(path, sep=';', encoding = "ISO-8859-1")

        # Remover todas as linhas com valor 'PENÚLTIMO' em 'ORDEM_EXERC'
        df = df[df['ORDEM_EXERC'] != 'PENÚLTIMO']

        df = df[['CD_CONTA', 'DS_CONTA']]

        if df_result.empty:
            df_result = df.copy()
        else:
            df_result = pd.concat([df_result, df], axis=0, ignore_index=True)
    #Converter para lower case
    df_result['DS_CONTA'] = df_result['DS_CONTA'].str.lower()
    df_result['DS_CONTA'] = df_result['DS_CONTA'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    df_result.set_index('CD_CONTA').T.to_dict('list')
    
    #Salvar relação CD_CONTA e DS_CONTA
    df_result.to_csv('../data_processed/'+itr_name+'_cd_conta_ds_conta.csv', encoding='utf-8', index=False)

c:\users\marina meyer\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:27: UserWarning: DataFrame columns are not unique, some columns will be omitted.
